# Работа с API через библиотеку requests
Для начала работы нужно [получить доступ](https://developers.sber.ru/docs/ru/gigachat/api/integration).
## Авторизация
Объявим ключи для авторизации

In [7]:
# Используем вкладку "Секреты" левой панели Colab для хранения ключей
from google.colab import userdata
client_id = userdata.get('SBER_ID')
secret = userdata.get('SBER_SECRET')
auth = userdata.get('SBER_AUTH')

Для авторизации можно использовать либо пару client_id и secret, но это требует некоторых действий

In [8]:
import base64
credentials = f"{client_id}:{secret}"
encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')

И получаемая в итоге строка идентична нашему ключу auth

In [9]:
encoded_credentials == auth

True

Поэтому используем для авторизации auth.

[Документация по авторизации](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/post-token)

In [10]:
import requests
import uuid

def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «GIGACHAT_API_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        # (можно скачать сертификаты Минцифры, тогда отключать проверку не надо)
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1


In [11]:
response = get_token(auth)
if response != 1:
  print(response.text)
  giga_token = response.json()['access_token']

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{"access_token":"eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.IUuexxOoPcyj1VyLjmPdEkXdoQQu9NGIdfu0bkar4T5GkGnCh6O6qu62nbZu9RF7PgmLc21EZmIpkdP5h2QNe2wd7swxKRe-gA3mw8CB5berrz4NtSiXsX1nvFtuttKMJK0EL8xx1ruewpF4QWwZwPv6gfqN2LHG1K9hKv9A-16Gd6OnA45Dgo-dqAGOuhMlw9jXbHCR4rG4izu4xMRzCWPGh7jUKElGpIoO0J2w5sDtU_LRADXj7KqF7fTYBE4l0_RMAY4UKOYvERtPgRXAsJPP2BLtlkXUPP4LmXRZE7FgAOU6RXs8wp7nd9dw081dNajO8tT7GMm5tEDVwlUdNA.cnBKjnDbpOxgian0p9U3CQ.cuRpZeYyZIhTgxdJW5kj6MW2PiJPjTeq4dChrxBfNeyuTskNs1ACYcR6dH9mZGFQX44IfsXOhSpqe7oG96uXvBd0ldIgtJ6F4vVVjSIXqbe1mHSERuWTrHovZQ9qZH43T_sU5oFCPLVgWlTxp_S2h7-PQ4yuAFJID3pf9EqyHiEiRDwLhU5XYJklwuuICX5oTzSbkWyNk9odmJ6XSNbmDRMrVXS-cyxydoFxDb8UUKgbJTtjwinxG2A_fdkFBoa7ivnm1a59aqMgYc002KIEl9m64Pwarph0vsyT8sM0J-VV6cCI0-ymdEMaJaaWKd3JcSsdStdi1ZVv42bV42v0zblWMMmVRxkqxjzToLP3q9UWZHoN3_iii_P-5mKbh_3VJlZ6Y7tivK2UU-DFolPvhcKgy5B5qaED_Hibj3ioCXaMdazbV2AmYMHoCMN7_NCqbLDf2bA5ABAzCCu8kAwZE9dVOROdAHTPmMbfWjI_qYEp8xo_COpbeUxbujhS407ye1qJm977zaYFBbVkhiG72nTKKxKm9

## Получение перечня моделей
[Документация](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/get-models)

In [12]:
import requests

url = "https://gigachat.devices.sberbank.ru/api/v1/models"

payload={}
headers = {
  'Accept': 'application/json',
  'Authorization': f'Bearer {giga_token}'
}

response = requests.request("GET", url, headers=headers, data=payload, verify=False)

print(response.text)

{"object":"list","data":[{"id":"GigaChat","object":"model","owned_by":"salutedevices"},{"id":"GigaChat-Plus","object":"model","owned_by":"salutedevices"},{"id":"GigaChat-Plus-preview","object":"model","owned_by":"salutedevices"},{"id":"GigaChat-Pro","object":"model","owned_by":"salutedevices"},{"id":"GigaChat-Pro-preview","object":"model","owned_by":"salutedevices"},{"id":"GigaChat-preview","object":"model","owned_by":"salutedevices"}]}


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Сделаем запросы на генерацию текста
Начнем с отправки отдельных сообщений

In [13]:
import requests
import json

def get_chat_completion(auth_token, user_message):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели GigaChat.

    Параметры:
    - auth_token (str): Токен для авторизации в API.
    - user_message (str): Сообщение от пользователя, для которого нужно получить ответ.

    Возвращает:
    - str: Ответ от API в виде текстовой строки.
    """
    # URL API, к которому мы обращаемся
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Подготовка данных запроса в формате JSON
    payload = json.dumps({
        "model": "GigaChat",  # Используемая модель
        "messages": [
            {
                "role": "user",  # Роль отправителя (пользователь)
                "content": user_message  # Содержание сообщения
            }
        ],
        "temperature": 1,  # Температура генерации
        "top_p": 0.1,  # Параметр top_p для контроля разнообразия ответов
        "n": 1,  # Количество возвращаемых ответов
        "stream": False,  # Потоковая ли передача ответов
        "max_tokens": 512,  # Максимальное количество токенов в ответе
        "repetition_penalty": 1,  # Штраф за повторения
        "update_interval": 0  # Интервал обновления (для потоковой передачи)
    })

    # Заголовки запроса
    headers = {
        'Content-Type': 'application/json',  # Тип содержимого - JSON
        'Accept': 'application/json',  # Принимаем ответ в формате JSON
        'Authorization': f'Bearer {auth_token}'  # Токен авторизации
    }

    # Выполнение POST-запроса и возвращение ответа
    try:
        response = requests.request("POST", url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        # Обработка исключения в случае ошибки запроса
        print(f"Произошла ошибка: {str(e)}")
        return -1



In [14]:
answer = get_chat_completion(giga_token, 'Как на Пайтоне сделать GET запрос?')

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [15]:
answer.json()

{'choices': [{'message': {'content': "Вы можете использовать модуль `requests` для выполнения GET-запросов на Python. Вот базовый пример:\n\n```python\nimport requests\n\nresponse = requests.get('http://example.com')\n\n```\n\nВ этом примере `response` будет содержать ответ на GET-запрос. Вы можете проверить статус кода ответа с помощью `response.status_code` и содержимое ответа с помощью `response.text`.",
    'role': 'assistant'},
   'index': 0,
   'finish_reason': 'stop'}],
 'created': 1714503590,
 'model': 'GigaChat:3.1.24.3',
 'object': 'chat.completion',
 'usage': {'prompt_tokens': 20,
  'completion_tokens': 93,
  'total_tokens': 113,
  'system_tokens': 204}}

In [16]:
print(answer.json()['choices'][0]['message']['content'])

Вы можете использовать модуль `requests` для выполнения GET-запросов на Python. Вот базовый пример:

```python
import requests

response = requests.get('http://example.com')

```

В этом примере `response` будет содержать ответ на GET-запрос. Вы можете проверить статус кода ответа с помощью `response.status_code` и содержимое ответа с помощью `response.text`.


In [17]:
from IPython.display import display, Markdown
display(Markdown(answer.json()['choices'][0]['message']['content']))

Вы можете использовать модуль `requests` для выполнения GET-запросов на Python. Вот базовый пример:

```python
import requests

response = requests.get('http://example.com')

```

В этом примере `response` будет содержать ответ на GET-запрос. Вы можете проверить статус кода ответа с помощью `response.status_code` и содержимое ответа с помощью `response.text`.

## Создадим диалог

In [18]:
import requests
import json

def get_chat_completion(auth_token, user_message, conversation_history=None):
    """
    Отправляет POST-запрос к API чата для получения ответа от модели GigaChat в рамках диалога.

    Параметры:
    - auth_token (str): Токен для авторизации в API.
    - user_message (str): Сообщение от пользователя, для которого нужно получить ответ.
    - conversation_history (list): История диалога в виде списка сообщений (опционально).

    Возвращает:
    - response (requests.Response): Ответ от API.
    - conversation_history (list): Обновленная история диалога.
    """
    # URL API, к которому мы обращаемся
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Если история диалога не предоставлена, инициализируем пустым списком
    if conversation_history is None:
        conversation_history = []

    # Добавляем сообщение пользователя в историю диалога
    conversation_history.append({
        "role": "user",
        "content": user_message
    })

    # Подготовка данных запроса в формате JSON
    payload = json.dumps({
        "model": "GigaChat:latest",
        "messages": conversation_history,
        "temperature": 1,
        "top_p": 0.1,
        "n": 1,
        "stream": False,
        "max_tokens": 512,
        "repetition_penalty": 1,
        "update_interval": 0
    })

    # Заголовки запроса
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json',
        'Authorization': f'Bearer {auth_token}'
    }

    # Выполнение POST-запроса и возвращение ответа
    try:
        response = requests.post(url, headers=headers, data=payload, verify=False)
        response_data = response.json()
        print(response_data)

        # Добавляем ответ модели в историю диалога
        conversation_history.append({
            "role": "assistant",
            "content": response_data['choices'][0]['message']['content']
        })

        return response, conversation_history
    except requests.RequestException as e:
        # Обработка исключения в случае ошибки запроса
        print(f"Произошла ошибка: {str(e)}")
        return None, conversation_history



In [19]:
# Пример использования функции для диалога

conversation_history = []

# Пользователь отправляет первое сообщение
response, conversation_history = get_chat_completion(giga_token, "Привет, как дела?", conversation_history)

# Пользователь отправляет следующее сообщение, продолжая диалог
response, conversation_history = get_chat_completion(giga_token, "Что ты умеешь делать?", conversation_history)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'choices': [{'message': {'content': 'Привет! Все хорошо, спасибо за интерес. Чем я могу вам помочь?', 'role': 'assistant'}, 'index': 0, 'finish_reason': 'stop'}], 'created': 1714503773, 'model': 'GigaChat:3.1.24.3', 'object': 'chat.completion', 'usage': {'prompt_tokens': 16, 'completion_tokens': 18, 'total_tokens': 34, 'system_tokens': 204}}


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'choices': [{'message': {'content': 'Я могу отвечать на вопросы, поддерживать беседу, предоставлять информацию, генерировать изображения и видео, а также выполнять арифметические действия.', 'role': 'assistant'}, 'index': 0, 'finish_reason': 'stop'}], 'created': 1714503775, 'model': 'GigaChat:3.1.24.3', 'object': 'chat.completion', 'usage': {'prompt_tokens': 47, 'completion_tokens': 31, 'total_tokens': 78, 'system_tokens': 204}}


In [20]:
conversation_history

[{'role': 'user', 'content': 'Привет, как дела?'},
 {'role': 'assistant',
  'content': 'Привет! Все хорошо, спасибо за интерес. Чем я могу вам помочь?'},
 {'role': 'user', 'content': 'Что ты умеешь делать?'},
 {'role': 'assistant',
  'content': 'Я могу отвечать на вопросы, поддерживать беседу, предоставлять информацию, генерировать изображения и видео, а также выполнять арифметические действия.'}]

## Добавим системный промпт
Системный промпт определяет роль, задавать контект и особенности общения бота с пользователем.

In [21]:
conversation_history = [{
    'role': 'system',
    'content': 'Отвечай как бывалый пират. Пусть тебя зовут Генри Морган.'

}

]

In [22]:
response, conversation_history = get_chat_completion(giga_token, 'Привет, друг!', conversation_history)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'choices': [{'message': {'content': 'Вечер добрый, капитан! Какими ветрами занесло в эти края?', 'role': 'assistant'}, 'index': 0, 'finish_reason': 'stop'}], 'created': 1714503898, 'model': 'GigaChat:3.1.24.3', 'object': 'chat.completion', 'usage': {'prompt_tokens': 36, 'completion_tokens': 20, 'total_tokens': 56, 'system_tokens': 21}}


In [23]:
response.json()['choices'][0]['message']['content']

'Вечер добрый, капитан! Какими ветрами занесло в эти края?'

## Генерация картинок Кандинским
[Документация](https://developers.sber.ru/docs/ru/gigachat/api/reference/rest/get-file-id)

Делаем запрос на генерацию прямо в GigaChat

In [24]:
import requests
import json

def send_chat_request(giga_token, user_message):
    """
    Отправляет POST-запрос к API GigaChat для получения ответа от модели чата.

    Параметры:
    - giga_token (str): Токен авторизации для доступа к API GigaChat.
    - user_message (str): Сообщение пользователя, которое будет обработано моделью GigaChat.

    Возвращает:
    - str: Строка сгенерированного ответа GigaChat с тэгом img
    """
    # URL API для отправки запросов к GigaChat
    url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

    # Заголовки для HTTP-запроса
    headers = {
        'Content-Type': 'application/json',  # Указываем, что отправляемые данные в формате JSON
        'Authorization': f'Bearer {giga_token}',  # Используем токен авторизации для доступа к API
    }

    # Данные для отправки в теле запроса
    payload = {
        "model": "GigaChat:latest",  # Указываем, что хотим использовать последнюю версию модели GigaChat
        "messages": [
            {
                "role": "user",  # Роль отправителя - пользователь
                "content": user_message  # Сообщение от пользователя
            },
        ],
        "temperature": 0.7  # Устанавливаем температуру, чтобы управлять случайностью ответов
    }

    try:
        # Отправляем POST-запрос к API и получаем ответ
        response = requests.post(url, headers=headers, data=json.dumps(payload), verify=False)
        # Выводим текст ответа. В реальных условиях следует обрабатывать ответ и проверять статус коды.
        print(response.json())
        return response.json()["choices"][0]["message"]["content"]
    except requests.RequestException as e:
        # В случае возникновения исключения в процессе выполнения запроса, выводим ошибку
        print(f"Произошла ошибка: {str(e)}")
        return None





In [25]:
user_message = "Нарисуй фотореалистичное изображение антропоморфного робота с \
ноутбуком в руках"
response_img_tag = send_chat_request(giga_token, user_message)
print(response_img_tag)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'choices': [{'message': {'content': 'Вот фотореалистичное изображение антропоморфного робота с ноутбуком в руках: <img src="c088ccef-a264-446a-b1ca-c4ece4bcac62" fuse="true"/>', 'role': 'assistant'}, 'index': 0, 'finish_reason': 'stop'}], 'created': 1714503954, 'model': 'GigaChat:3.1.24.3', 'object': 'chat.completion', 'usage': {'prompt_tokens': 32, 'completion_tokens': 44, 'total_tokens': 76, 'system_tokens': 204}}
Вот фотореалистичное изображение антропоморфного робота с ноутбуком в руках: <img src="c088ccef-a264-446a-b1ca-c4ece4bcac62" fuse="true"/>


Получаем само изображение и сохраняем его в файл

In [26]:
from bs4 import BeautifulSoup



# Парсим HTML
soup = BeautifulSoup(response_img_tag, 'html.parser')

# Извлекаем значение атрибута `src`
img_src = soup.img['src']

print(img_src)

c088ccef-a264-446a-b1ca-c4ece4bcac62


In [27]:
import requests

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {giga_token}',
}

response = requests.get(f'https://gigachat.devices.sberbank.ru/api/v1/files/{img_src}/content', headers=headers, verify=False)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [28]:
with open('image.jpg', 'wb') as f:
    f.write(response.content)